In [2]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

In [3]:
PATH = Path('../input/optiver-realized-volatility-prediction')


In [29]:
df_train = pd.read_csv(PATH/'train_with_ftrs.csv').drop(['row_id','stock_id_target_enc'], axis=1)

In [30]:
kf = KFold(n_splits=5, random_state=19901028, shuffle=True)
trn_idx, val_idx = first(kf.split(df_train))

# Neural Net

In [68]:
df_train.stock_id = df_train.stock_id.astype('category')

cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

procs_nn = [Categorify, FillMissing, Normalize]
to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                      splits=[list(trn_idx), list(val_idx)], y_names='target')

dls = to_nn.dataloaders(2048)

In [69]:
learn = tabular_learner(dls, y_range=(0,.1), layers=[400,200,100],
                        n_out=1)

In [70]:
learn.load('tabular226.pth')

In [71]:
preds, tars = learn.get_preds()

In [72]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [73]:
preds.shape, tars.shape

(torch.Size([85787, 1]), torch.Size([85787, 1]))

In [74]:
rmspe(preds, tars)

tensor(0.2268)

# Test Set

In [75]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

In [76]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [77]:
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

In [78]:
def count_unique(series):
    return len(np.unique(series))

In [79]:
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    #calculate return etc
    df['wap'] = calc_wap(df)
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    
    df['wap2'] = calc_wap2(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_balance'] = abs(df['wap'] - df['wap2'])
    
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    #dict for aggregate
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap_balance':[np.mean],
        'price_spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap':[np.mean],
            }

    #####groupby / all seconds
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second 
    
        df_feature_sec = pd.DataFrame(df.query(f'seconds_in_bucket >= {second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()

        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns] #time_id is changed to time_id_
     
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))

        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    #create row_id
    stock_id = str(file_path).split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature

In [80]:
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    ######groupby / last XX seconds
    last_seconds = [300]
    
    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = str(file_path).split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [81]:
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = PATH / f"book_train.parquet/stock_id={stock_id}"
            file_path_trade = PATH / f"trade_train.parquet/stock_id={stock_id}"
        else:
            file_path_book = PATH / f"book_test.parquet/stock_id={stock_id}"
            file_path_trade = PATH / f"trade_test.parquet/stock_id={stock_id}" 
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [82]:
test = pd.read_csv(PATH/'test.csv')
test_ids = test.stock_id.unique()
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished


In [83]:
df_test=df_test.fillna(0)

In [84]:
df_test = df_test.drop(['row_id', 'time_id'], axis=1)

In [85]:
test_dl = dls.test_dl(df_test)

In [86]:
preds,_ = learn.get_preds(dl=test_dl)

In [87]:
preds

tensor([[0.],
        [0.],
        [0.]])

In [89]:
btest_dl.one_batch())

2

In [55]:
df_test

,stock_id,time_id,row_id,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,...,total_volume_mean_300,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300
0,0,4,0-4,0.000294,0.000252,0.000145,0.000557,0.000393,-0.000115,164.666667,...,0.0,0.0,0.000295,3.0,201.0,3.666667,0.0,0.0,0.0,0.0
1,0,32,0-32,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0,34,0-34,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [56]:
df_train

,target,log_return_realized_volatility,log_return2_realized_volatility,wap_balance_mean,price_spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,...,wap_mean_300,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,stock_id
0,0.004136,0.004499,0.006999,0.000388,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,...,1.003753,0.002006,40.0,3179.0,2.750000,0.001308,21.0,1587.0,2.571429,0
1,0.001445,0.001204,0.002476,0.000212,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,...,1.000397,0.000901,30.0,1289.0,1.900000,0.000587,16.0,900.0,2.250000,0
2,0.002168,0.002369,0.004801,0.000331,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,...,0.998685,0.001961,25.0,2161.0,2.720000,0.001137,12.0,1189.0,3.166667,0
3,0.002195,0.002574,0.003637,0.000380,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,...,0.998436,0.001561,15.0,1962.0,3.933333,0.001089,9.0,1556.0,5.111111,0
4,0.001747,0.001894,0.003257,0.000254,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,...,0.999488,0.000871,22.0,1791.0,4.045455,0.000453,11.0,1219.0,4.909091,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428927,0.003461,0.003691,0.005876,0.000361,0.000878,0.000091,-0.000202,161.638710,406.045161,0.999582,...,0.999375,0.002171,37.0,2570.0,2.783784,0.001451,18.0,796.0,2.055556,126
428928,0.003113,0.004104,0.004991,0.000295,0.000706,0.000126,-0.000142,150.578475,243.322870,1.002476,...,1.003528,0.002180,43.0,2323.0,3.418605,0.001791,20.0,1107.0,3.550000,126
428929,0.004070,0.003118,0.006019,0.000394,0.000739,0.000189,-0.000192,254.406250,348.093750,1.001082,...,1.001282,0.001921,35.0,3740.0,2.800000,0.001580,24.0,2750.0,2.541667,126
428930,0.003357,0.003661,0.005362,0.000231,0.000530,0.000143,-0.000134,145.654135,426.416040,1.001809,...,1.001807,0.002051,80.0,9389.0,2.925000,0.001520,43.0,5150.0,2.813953,126


In [67]:
len(df_test.columns)

27

In [66]:
len(df_train.columns)

28